In [1]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
import traceback

# Define paths and project range
BASE_DIRECTORY = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data"
OUTPUT_CSV_PATH_ORIGINAL = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 14/03_raw/ph2_rawdata_cluster14_style_G_originals.csv"
OUTPUT_CSV_PATH_ADDENDUM = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 14/03_raw/ph2_rawdata_cluster14_style_G_addendums.csv"
LOG_FILE_PATH = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_2_cost_data/Cluster 14/03_raw/ph2_scraping_cluster14_style_G_log.txt"
PROJECT_RANGE = range(1831, 2193)  # Inclusive range for q_ids in Clusters 11

# Initialize DataFrames
core_originals = pd.DataFrame()
core_addendums = pd.DataFrame()

# Initialize tracking variables
scraped_projects = set()
skipped_projects = set()
missing_projects = set()
scraped_pdfs = []
skipped_pdfs = []
addendum_pdfs = []
original_pdfs = []
style_n_pdfs = []  # List to track style N PDFs
total_pdfs_accessed = 0
total_pdfs_scraped = 0
total_pdfs_skipped = 0
original_has_table7 = {}  # Dictionary to track if original PDFs have table7

def clean_column_headers(headers):
    """Cleans column headers by normalizing and removing unwanted characters."""
    cleaned_headers = []
    for header in headers:
        if header is None:
            header = ""
        elif isinstance(header, str):
            header = header.lower()
            header = re.sub(r'\s+', ' ', header)
            header = re.sub(r'\(.*?\)', '', header)
            header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
            header = header.strip()
        cleaned_headers.append(header)
    return cleaned_headers

def clean_string_cell(value):
    """Cleans string cells by removing newlines and trimming spaces."""
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()
    return value

def contains_phrase(row, phrase):
    """Checks if any cell in a row contains a specific phrase."""
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

def extract_specific_phrase(title):
    """
    Extracts a specific phrase from the table title based on predefined keywords.

    Args:
        title (str): The table title string.

    Returns:
        str: The extracted specific phrase if found, else the original title.
    """
    phrases = [
        "PTO",
        "Reliability Network Upgrade",
        "Area Delivery Network Upgrade",
        "Local Delivery Network",
        "Other Potential Network Upgrade",
        "Area Delivery Network Upgrades",
        "Conditionally Assigned Network Upgrades",
        "ADNU",
        "LDNU",
        "RNU"
    ]

    for phrase in phrases:
        if re.search(rf"\b{re.escape(phrase)}\b", title, re.IGNORECASE):
            return phrase
    return title  # Fallback to the entire title if no specific phrase is found

def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type of upgrade",
        "upgrade",
        "description",
        "cost allocation factor"
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df

def search_gps_coordinates(text, log_file):
    """Search for GPS coordinates using multiple patterns."""
    gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
    if gps_coords:
        print(f"Found GPS coordinates: {gps_coords.groups()}", file=log_file)
        return gps_coords.groups()

    project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
    if project_coords:
        print(f"Found project coordinates: {project_coords.groups()}", file=log_file)
        return project_coords.groups()

    gps_coords_directional = re.search(
        r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
    if gps_coords_directional:
        lat, lon = gps_coords_directional.groups()
        latitude = lat if "N" in text.upper() else f"-{lat}"  # Adjust latitude sign
        longitude = lon if "E" in text.upper() else f"-{lon}"  # Adjust longitude sign
        print(f"Found directional GPS coordinates: {(latitude, longitude)}", file=log_file)
        return (latitude, longitude)

    print("GPS coordinates not found.", file=log_file)
    return (None, None)

def extract_table1(pdf_path, log_file):
    """
    Extracts the Point of Interconnection from Table 1 in the provided PDF.
    Implements a retry mechanism with different table extraction settings if initial attempts fail.

    Args:
        pdf_path (str): Path to the PDF file.
        log_file (file object): Log file to write print statements.

    Returns:
        str: Extracted Point of Interconnection value,
             "Value Missing" if label found but no value,
             or None if not found.
    """
    print(f"\nProcessing {pdf_path} for Table 1 extraction...", file=log_file)
    point_of_interconnection = None

    # Define the regex pattern for 'Point of Interconnection' (case-insensitive)
    poi_pattern = re.compile(r"Point\s+of\s+Interconnection", re.IGNORECASE)

    # Define different table extraction settings to try
    table_settings_list = [
        {
            "horizontal_strategy": "text",
            "vertical_strategy": "lines",
            "snap_tolerance": 1,
        },
        {
            "horizontal_strategy": "lines",
            "vertical_strategy": "lines",
            "snap_tolerance": 2,  # Increased tolerance for retry
        }
    ]

    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Identify all pages that contain "Table 1"
            table1_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*1\b", text, re.IGNORECASE):
                    table1_pages.append(i)

            if not table1_pages:
                print("No Table 1 found in the PDF.", file=log_file)
                return None  # Return None if no Table 1 found

            first_page = table1_pages[0]
            last_page = table1_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 2  # Plus one to include the next page if needed

            print(f"Table 1 starts on page {scrape_start + 1} and ends on page {scrape_end + 1}", file=log_file)

            # Flag to indicate if extraction was successful
            extraction_successful = False

            # Iterate through the specified page range
            for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1} for Table 1...", file=log_file)

                for attempt, table_settings in enumerate(table_settings_list, start=1):
                    print(f"\nAttempt {attempt} with table settings: {table_settings}", file=log_file)
                    tables = page.find_tables(table_settings=table_settings)
                    print(f"Found {len(tables)} table(s) on page {page_number + 1} with current settings.", file=log_file)

                    for table_index, table in enumerate(tables, start=1):
                        tab = table.extract()
                        if not tab:
                            print(f"Table {table_index} on page {page_number + 1} is empty. Skipping.", file=log_file)
                            continue  # Skip empty tables

                        print(f"\n--- Table {table_index} on Page {page_number + 1} ---", file=log_file)
                        for row_num, row in enumerate(tab, start=1):
                            print(f"Row {row_num}: {row}", file=log_file)

                        # Iterate through each row in the table
                        for row_index, row in enumerate(tab, start=1):
                            # Iterate through each cell in the row
                            for cell_index, cell in enumerate(row, start=1):
                                if cell and poi_pattern.search(cell):
                                    # Assuming the next column contains the value
                                    poi_col_index = cell_index  # 1-based index
                                    adjacent_col_index = poi_col_index + 1  # Next column

                                    if adjacent_col_index <= len(row):
                                        poi_value = clean_string_cell(row[adjacent_col_index - 1])
                                        if poi_value:  # Check if the value is not empty
                                            point_of_interconnection = poi_value
                                            print(f"\nFound Point of Interconnection: '{point_of_interconnection}' "
                                                  f"(Page {page_number + 1}, Table {table_index}, Row {row_index})", file=log_file)
                                            extraction_successful = True
                                            break  # Exit the cell loop
                                        else:
                                            print(f"\nPoint of Interconnection label found but adjacent value is empty "
                                                  f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                            # Proceed to scan surrounding rows for the value
                                            poi_value_parts = []

                                            # Define the range to scan: two rows above and two rows below
                                            # Convert to 0-based index
                                            current_row_idx = row_index - 1
                                            start_scan = max(0, current_row_idx - 2)
                                            end_scan = min(len(tab), current_row_idx + 3)  # Exclusive

                                            print(f"Scanning rows {start_scan + 1} to {end_scan} for POI value parts.", file=log_file)

                                            for scan_row_index in range(start_scan, end_scan):
                                                # Skip the current row where the label was found
                                                if scan_row_index == current_row_idx:
                                                    continue

                                                scan_row = tab[scan_row_index]
                                                # Ensure the adjacent column exists in the scan row
                                                if adjacent_col_index - 1 < len(scan_row):
                                                    scan_cell = clean_string_cell(scan_row[adjacent_col_index - 1])
                                                    if scan_cell and not poi_pattern.search(scan_cell):
                                                        poi_value_parts.append(scan_cell)
                                                        print(f"Found POI part in row {scan_row_index + 1}: '{scan_cell}'", file=log_file)
                                                    elif poi_pattern.search(scan_cell):
                                                        # If another POI label is found, skip it
                                                        print(f"Encountered another POI label in row {scan_row_index + 1}. Skipping this row.", file=log_file)
                                                        continue

                                            if poi_value_parts:
                                                # Concatenate the parts to form the complete POI value
                                                point_of_interconnection = " ".join(poi_value_parts)
                                                print(f"\nConcatenated Point of Interconnection: '{point_of_interconnection}' "
                                                      f"(Page {page_number + 1}, Table {table_index})", file=log_file)
                                                extraction_successful = True
                                                break  # Exit the cell loop
                                            else:
                                                print(f"\nNo POI value found in the surrounding rows "
                                                      f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                                # Do not return immediately; proceed to retry
                                    else:
                                        print(f"\nPoint of Interconnection label found but no adjacent column "
                                              f"(Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                        # Do not return immediately; proceed to retry
                            if extraction_successful:
                                break  # Exit the row loop
                        if extraction_successful:
                            break  # Exit the table loop
                    if extraction_successful:
                        break  # Exit the attempt loop
                if extraction_successful:
                    break  # Exit the page loop

    except Exception as e:
        print(f"Error processing Table 1 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return None

    if not extraction_successful:
        # After all attempts, determine the appropriate return value
        if point_of_interconnection is not None and point_of_interconnection != "":
            # Label was found but no value
            print("Point of Interconnection label found but no adjacent value.", file=log_file)
            return "Value Missing"
        else:
            # Label not found
            print("Point of Interconnection not found in Table 1.", file=log_file)
            return None

    return point_of_interconnection

def extract_base_data(pdf_path, project_id, log_file):
    """Extract base data from the PDF and return as a DataFrame."""
    print("Extracting base data from PDF...", file=log_file)
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text

        text = clean_string_cell(text)

        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id = queue_id.group(1) if queue_id else str(project_id)  # Use project_id if queue_id is not found
        print(f"Extracted Queue ID: {queue_id}", file=log_file)

        # Updated Cluster Extraction
        clusters = re.findall(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        if '14' in clusters:
            cluster_number = '14'
        elif clusters:
            cluster_number = max(clusters, key=lambda x: int(x))  # Choose the highest cluster number found
        else:
            cluster_number = '14'  # Default to 12 if not found
        print(f"Extracted Cluster Number: {cluster_number}", file=log_file)

        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None
        print(f"Extracted Deliverability Status: {deliverability_status}", file=log_file)

        # Extract Capacity
        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None
        print(f"Extracted Capacity: {capacity}", file=log_file)

        # Extract Point of Interconnection
        point_of_interconnection = extract_table1(pdf_path, log_file)

        latitude, longitude = search_gps_coordinates(text, log_file)

        # Initialize base data dictionary
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [point_of_interconnection]
        }

        print("Base data extracted:", file=log_file)
        print(base_data, file=log_file)
        return pd.DataFrame(base_data)

    except Exception as e:
        print(f"Error extracting base data from {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()  # Return empty DataFrame on error

def adjust_rows_length(data_rows, headers):
    """Ensure each row in data_rows matches the length of headers by truncating or padding."""
    col_count = len(headers)
    for i in range(len(data_rows)):
        row = data_rows[i]
        if len(row) > col_count:
            data_rows[i] = row[:col_count]
        elif len(row) < col_count:
            data_rows[i].extend([""]*(col_count - len(row)))

def extract_table7(pdf_path, log_file, is_addendum=False):
    """
    Extracts Table 7 data from the provided PDF.

    Args:
        pdf_path (str): Path to the PDF file.
        log_file (file object): Log file to write print statements.
        is_addendum (bool): Whether the PDF is an addendum.

    Returns:
        pd.DataFrame: Extracted Table 7 data.
    """
    print(f"\nProcessing {pdf_path} for Table 7 extraction...", file=log_file)
    extracted_tables = []
    specific_phrase = None

    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Identify pages that contain "Table 7-1" to "Table 7-5" with hyphen or dot
            table7_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if  re.search(r"Table\s*7[-.]([2-6])\b", text, re.IGNORECASE): #re.search(r"(Modification\s+of\s+)?Table\s*7[-.]([1-5])\b", text, re.IGNORECASE):
                    table7_pages.append(i)


                     


            if not table7_pages:
                print("No Table 7-1 to 7-5 found in the PDF.", file=log_file)
                return pd.DataFrame()

            first_page = table7_pages[0]
            last_page = table7_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 2

            print(f"Table 7 starts on page {scrape_start + 1} and ends on page {scrape_end}", file=log_file)

            for page_number in range(scrape_start, min(scrape_end, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1}...", file=log_file)
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })

                for table_index, table in enumerate(tables):
                    tab = table.extract()
                    if not tab:
                        print(f"Table {table_index + 1} on page {page_number + 1} is empty. Skipping.", file=log_file)
                        continue

                    table_bbox = table.bbox
                    title_bbox = (0, 0, page.width, table_bbox[1])
                    title_text = page.within_bbox(title_bbox).extract_text() or ""
                    table_title = None

                    if title_text:
                        title_lines = title_text.split('\n')[::-1]
                        for line in title_lines:
                            line = line.strip()
                            match = re.search(r"(Modification\s+of\s+)?Table\s*7[-.]([1-5])[:\-\s]*(.*)", line, re.IGNORECASE)
                            if match:
                                table_title = match.group(3).strip()
                                break

                    if table_title:
                        # New Table 7 detected
                        specific_phrase = extract_specific_phrase(table_title)
                        print(f"New Table 7 detected: '{specific_phrase}' on page {page_number + 1}, table {table_index + 1}", file=log_file)

                        headers = clean_column_headers(tab[0])
                        data_rows = tab[1:]

                        # Create DataFrame for new table
                        try:
                            df_new = pd.DataFrame(data_rows, columns=headers)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for new table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                            continue

                        # Handle new ADNU tables (grouping logic)
                        if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                            print("Detected 'Area Delivery Network Upgrade' table (new).", file=log_file)
                            if "adnu" in df_new.columns:
                                if "type of upgrade" not in df_new.columns:
                                    # Group all adnu rows into one 'upgrade' row
                                    adnu_values = df_new["adnu"].dropna().astype(str).tolist()
                                    grouped_adnu = " ".join(adnu_values)
                                    other_columns = df_new.drop(columns=["adnu"]).iloc[0].to_dict()

                                    df_grouped = pd.DataFrame({
                                        "upgrade": [grouped_adnu],
                                        "type of upgrade": [specific_phrase]
                                    })

                                    for col, value in other_columns.items():
                                        df_grouped[col] = value

                                    print("Grouped all 'adnu' rows into a single 'upgrade' row for new ADNU table.", file=log_file)
                                    df_new = df_grouped
                                else:
                                    # If 'type of upgrade' exists, just rename adnu if needed
                                    if "upgrade" in df_new.columns:
                                        df_new.drop(columns=['adnu'], inplace=True)
                                        print("Dropped 'adnu' column to avoid duplicate 'upgrade'.", file=log_file)
                                    else:
                                        df_new.rename(columns={'adnu': 'upgrade'}, inplace=True)
                                        print("Renamed 'adnu' to 'upgrade' in new ADNU table.", file=log_file)
                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' to all rows in new ADNU table.", file=log_file)
                            else:
                                # If 'type of upgrade' exists and first row none, original logic replaced only first row if needed
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    df_new.at[0, "type of upgrade"] = specific_phrase
                                    print("Replaced None in 'type of upgrade' first row for new ADNU table.", file=log_file)
                        else:
                            # Non-ADNU new tables
                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' to all rows in new non-ADNU table.", file=log_file)
                            else:
                                # If exist and none in first row original logic replaced only first row
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print("Replacing None in 'type of upgrade' for the first row in new non-ADNU table.", file=log_file)
                                    df_new.at[0, "type of upgrade"] = specific_phrase

                        if df_new.columns.duplicated().any():
                            print("Duplicate columns detected in new table. Dropping duplicates.", file=log_file)
                            df_new = df_new.loc[:, ~df_new.columns.duplicated()]

                        extracted_tables.append(df_new)
                    else:
                        # Continuation Table
                        if specific_phrase is None:
                            print(f"No previous Table 7 title found for continuation on page {page_number + 1}, table {table_index + 1}. Skipping.", file=log_file)
                            continue

                        print(f"Continuation Table detected on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        data_rows = tab

                        # Check if the number of columns matches
                        expected_columns = len(extracted_tables[-1].columns) if extracted_tables else None
                        if expected_columns is None:
                            print(f"No existing table to continue with on page {page_number + 1}, table {table_index + 1}. Skipping.", file=log_file)
                            continue  # No table to continue with

                        # Define expected columns based on the last extracted table
                        expected_headers = extracted_tables[-1].columns.tolist()

                        # Detect header row in continuation
                        header_keywords = ["type of upgrade", "adnu", "MW at POI"]
                        first_continuation_row = data_rows[0] if data_rows else []
                        is_header_row = any(
                            re.search(rf"\b{kw}\b", str(cell), re.IGNORECASE) for kw in header_keywords for cell in first_continuation_row
                        )

                        if is_header_row:
                            print(f"Detected header row in continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                            actual_header_row = data_rows[0]
                            actual_headers = clean_column_headers(actual_header_row)

                            # Go back to previous page bounding box to update table title if needed
                            if page_number > 0:
                                previous_page = pdf.pages[page_number - 1]
                                bbox_lower_region = (0, table_bbox[1], previous_page.width, previous_page.height)
                                title_text_previous = previous_page.within_bbox(bbox_lower_region).extract_text() or ""
                                new_table_title = None
                                if title_text_previous:
                                    title_lines_prev = title_text_previous.split('\n')[::-1]
                                    for line in title_lines_prev:
                                        line = line.strip()
                                        match_prev = re.search(r"(Modification\s+of\s+)?Table\s*7[-.]([1-5])[:\-\s]*(.*)", line, re.IGNORECASE)
                                        if match_prev:
                                            new_table_title = match_prev.group(3).strip()
                                            break
                                if new_table_title:
                                    specific_phrase = extract_specific_phrase(new_table_title)
                                    print(f"Updated table title from previous page: '{specific_phrase}' for continuation table on page {page_number + 1}, table {table_index + 1}", file=log_file)
                                else:
                                    print("No table title found in previous page region. Using existing specific_phrase.", file=log_file)
                            else:
                                print("No previous page available for title extraction for continuation table. Using existing specific_phrase.", file=log_file)

                            # Handle continuation ADNU or non-ADNU
                            if re.search(r"Area\s*Delivery\s*Network\s*Upgrades", specific_phrase, re.IGNORECASE):
                                print("Continuation ADNU table detected. No grouping, just rename and type of upgrade handling.", file=log_file)
                                if "type of upgrade" not in data_rows[0]:
                                    # Insert 'type of upgrade' column at the beginning
                                    print("Inserting 'type of upgrade' column with specific phrase in continuation table.", file=log_file)
                                    data_rows = [ [specific_phrase] + row for row in data_rows ]

                                    if "ADNU" in data_rows[0]:
                                        print("Handling continuation for 'Area Delivery Network Upgrade' table", file=log_file)
                                        print("Renaming 'ADNU' to 'upgrade' in continuation table.", file=log_file)
                                        # Find the index where "ADNU" occurs in the first row
                                        adnu_idx = data_rows[0].index("ADNU")
                                        # Replace "ADNU" with "upgrade" in that column for every row
                                        data_rows[0][adnu_idx] = "upgrade"
                            else:
                                # General Handling for other tables
                                if "type of upgrade" not in data_rows[0]:
                                    # Insert 'type of upgrade' column at the beginning
                                    print("Inserting 'type of upgrade' column with specific phrase in continuation table.", file=log_file)
                                    data_rows = [ [specific_phrase] + row for row in data_rows ]

                                    if "Type of Upgrade" in data_rows[0]:
                                        print("Handling continuation for non-ADNU table.", file=log_file)
                                        print("Renaming 'Type of Upgrade' to 'upgrade' in continuation table.", file=log_file)
                                        # Find the index where "Type of Upgrade" occurs in the first row
                                        upgrade_idx = data_rows[0].index("Type of Upgrade")
                                        # Replace "Type of Upgrade" with "upgrade" in that column for every row
                                        data_rows[0][upgrade_idx] = "upgrade"

                        # Handle missing or extra columns
                        if len(data_rows[0]) < expected_columns:
                            if re.search(r"Area\s*Delivery\s*Network\s*Upgrade", specific_phrase, re.IGNORECASE):
                                # For ADNU tables, assume missing "upgrade" column
                                print(f"Detected missing 'upgrade' column in continuation table on page {page_number + 1}, table {table_index + 1}. Inserting 'upgrade' column.", file=log_file)
                                data_rows = [row[:7] + [specific_phrase] + row[7:] for row in data_rows]

                            else:
                                # For other tables, assume missing "Type of Upgrade" column
                                print(f"Detected missing 'Type of Upgrade' column in continuation table on page {page_number + 1}, table {table_index + 1}. Inserting 'Type of Upgrade' column.", file=log_file)
                                data_rows = [[specific_phrase] + row for row in data_rows]
                        elif len(data_rows[0]) > expected_columns:
                            # Extra columns detected; adjust accordingly
                            print(f"Detected extra columns in continuation table on page {page_number + 1}, table {table_index + 1}. Dropping extra columns.", file=log_file)
                            data_rows = [row[:expected_columns] for row in data_rows]

                        # Create DataFrame for the continuation table
                        if is_header_row:
                            data_rows = data_rows[1:]
                            print(f"Dropped header row from data_rows after modifications for continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)

                        try:
                            df_continuation = pd.DataFrame(data_rows, columns=expected_headers)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for continuation table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                            continue  # Skip this table due to error

                        # Special Handling for "Area Delivery Network Upgrade" Tables in Continuation
                        if re.search(r"Area\s*Delivery\s*Network\s*Upgrades", specific_phrase, re.IGNORECASE):
                            if "type of upgrade" in df_continuation.columns:
                                first_row = df_continuation.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print(f"Replacing 'None' in 'type of upgrade' for the first data row of continuation on page {page_number + 1}, table {table_index + 1}", file=log_file)
                                    df_continuation.at[0, "type of upgrade"] = specific_phrase
                            else:
                                # If "type of upgrade" column does not exist, add it
                                df_continuation["type of upgrade"] = specific_phrase
                                print(f"'type of upgrade' column added with value '{specific_phrase}' for continuation on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        else:
                            # General Handling for other tables
                            if "type of upgrade" in df_continuation.columns:
                                first_row = df_continuation.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print(f"Replacing 'None' in 'Type of Upgrade' for the first data row of continuation on page {page_number + 1}, table {table_index + 1}", file=log_file)
                                    df_continuation.at[0, "type of upgrade"] = specific_phrase
                            else:
                                # If "Type of Upgrade" column does not exist, add it
                                df_continuation["type of upgrade"] = specific_phrase
                                print(f"'Type of Upgrade' column added with value '{specific_phrase}' for continuation on page {page_number + 1}, table {table_index + 1}", file=log_file)

                        # Ensure no duplicate columns
                        if df_continuation.columns.duplicated().any():
                            print(f"Duplicate columns detected in continuation table on page {page_number + 1}, table {table_index + 1}. Dropping duplicates.", file=log_file)
                            df_continuation = df_continuation.loc[:, ~df_continuation.columns.duplicated()]

                        # Merge with the last extracted table
                        extracted_tables[-1] = pd.concat([extracted_tables[-1], df_continuation], ignore_index=True, sort=False)            

    except Exception as e:
        print(f"Error processing Table 7 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()

    # After processing all tables, concatenate them
    if extracted_tables:
        all_columns = set()
        for df in extracted_tables:
            all_columns.update(df.columns.tolist())

        standardized_tables = []
        for df in extracted_tables:
            standardized_df = df.reindex(columns=all_columns)
            standardized_tables.append(standardized_df)

        print("\nConcatenating all extracted Table 7 data...", file=log_file)
        try:
            table7_data = pd.concat(standardized_tables, ignore_index=True, sort=False)
            print(f"Successfully concatenated {len(standardized_tables)} tables.", file=log_file)
        except Exception as e:
            print(f"Error concatenating tables: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            table7_data = pd.DataFrame()
    else:
        print("No Table 7 data extracted.", file=log_file)
        table7_data = pd.DataFrame()

    return table7_data

def scrape_alternative_tables(pdf_path, log_file, specific_phrases):
    """
    Scrapes all tables in the PDF that contain any of the specific phrases in their cells.

    Args:
        pdf_path (str): Path to the PDF file.
        log_file (file object): Log file to write print statements.
        specific_phrases (list): List of specific phrases to search for.

    Returns:
        pd.DataFrame: Extracted data from tables containing the specific phrases.
    """
    print(f"\nProcessing {pdf_path} for alternative table scraping...", file=log_file)
    extracted_tables = []

    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })
                for table_index, table in enumerate(tables, start=1):
                    tab = table.extract()
                    if not tab:
                        continue
                    # Check if any cell contains any of the specific phrases
                    match_found = False
                    for row in tab:
                        for cell in row:
                            if cell and any(re.search(rf"\b{re.escape(phrase)}\b", cell, re.IGNORECASE) for phrase in specific_phrases):
                                match_found = True
                                break
                        if match_found:
                            break
                    if match_found:
                        headers = clean_column_headers(tab[0])
                        headers = make_unique_headers(headers)  # Ensure headers are unique
                        data_rows = tab[1:]
                        try:
                            df = pd.DataFrame(data_rows, columns=headers)
                            extracted_tables.append(df)
                            print(f"Extracted alternative table from page {page_number}, table {table_index}.", file=log_file)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for alternative table on page {page_number}, table {table_index}: {ve}", file=log_file)
                            continue

    except Exception as e:
        print(f"Error scraping alternative tables in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()

    if extracted_tables:
        all_columns = set()
        for df in extracted_tables:
            all_columns.update(df.columns.tolist())

        standardized_tables = []
        for df in extracted_tables:
            # Ensure columns are unique before reindexing
            standardized_df = df.reindex(columns=all_columns)
            standardized_tables.append(standardized_df)

        print("\nConcatenating all extracted alternative tables...", file=log_file)
        try:
            alternative_data = pd.concat(standardized_tables, ignore_index=True, sort=False)
            print(f"Successfully concatenated {len(standardized_tables)} alternative tables.", file=log_file)
            return alternative_data
        except Exception as e:
            print(f"Error concatenating alternative tables: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            return pd.DataFrame()
    else:
        print("No alternative tables found containing specific phrases.", file=log_file)
        return pd.DataFrame()


def extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False):
    """Extracts Table 7 data and merges with base data."""
    base_data = extract_base_data(pdf_path, project_id, log_file)
    table7_data = extract_table7(pdf_path, log_file, is_addendum)

    if table7_data.empty:
        if is_addendum and original_has_table7.get(project_id, False):
            # Scrape alternative tables based on specific phrases
            specific_phrases = [
                "PTO",
                "Reliability Network Upgrade",
                "Area Delivery Network Upgrade",
                "Local Delivery Network",
                "Other Potential Network Upgrade",
                "Area Delivery Network Upgrades",
                "Conditionally Assigned Network Upgrades",
                "ADNU",
                "LDNU",
                "RNU"
            ]
            alternative_data = scrape_alternative_tables(pdf_path, log_file, specific_phrases)
            if not alternative_data.empty:
                table7_data = alternative_data
    if table7_data.empty:
        return base_data
    else:
        # Identify overlapping columns excluding 'point_of_interconnection'
        overlapping_columns = base_data.columns.intersection(table7_data.columns).difference(['point_of_interconnection'])
        table7_data = table7_data.drop(columns=overlapping_columns, errors='ignore')
        
        # Repeat base data for each row in table7_data
        base_data_repeated = pd.concat([base_data] * len(table7_data), ignore_index=True)
        
        try:
            # Concatenate base data with Table 7 data along columns
            merged_df = pd.concat([base_data_repeated, table7_data], axis=1, sort=False)
            
            # Ensure 'point_of_interconnection' is present and correctly populated
            if 'point_of_interconnection' not in merged_df.columns:
                merged_df['point_of_interconnection'] = base_data['point_of_interconnection'].iloc[0]
                print(f"Added 'point_of_interconnection' to merged data for {pdf_path}.", file=log_file)
            
            print(f"Merged base data with Table 7 data for {pdf_path}.", file=log_file)
            return merged_df
        except Exception as e:
            print(f"Error merging base data with Table 7 data for {pdf_path}: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            return base_data  # Fallback to base data only

def check_has_table7(pdf_path):
    """Checks if the PDF contains Table 7-1 to 7-5."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text() or ""
                if re.search(r"(Modification\s+of\s+)?Table\s*7[-.]([2-6])\b", text, re.IGNORECASE):
                    return True
    except Exception as e:
        # Handle potential errors when opening PDF
        return False
    return False

def has_network_upgrade_type_column(pdf_path, log_file):
    """Checks if any table in the PDF has a column header 'Network Upgrade Type'."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page_number, page in enumerate(pdf.pages, start=1):
                tables = page.find_tables()
                for table_index, table in enumerate(tables, start=1):
                    tab = table.extract()
                    if not tab:
                        continue
                    headers = clean_column_headers(tab[0])
                    if "network upgrade type" in headers:
                        print(f"Found 'Network Upgrade Type' in PDF {pdf_path} on page {page_number}, table {table_index}.", file=log_file)
                        return True
    except Exception as e:
        print(f"Error checking 'Network Upgrade Type' in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
    return False

def is_addendum(pdf_path, log_file):
    """Checks if the PDF is an addendum by searching 'Addendum' or 'Revision' on the first page."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if len(pdf.pages) == 0:
                return False
            first_page = pdf.pages[0]
            text = first_page.extract_text() or ""
            print(f"Extracted Text: {text}", file= log_file)  # Debugging step
            # Case-insensitive check for 'Addendum' or 'Revision'
            text_lower = text.lower()
            return "addendum" in text_lower or "revision" in text_lower
    except Exception as e:
        # Handle potential errors when opening PDF
        return False



def make_unique_headers(headers):
    """
    Appends a suffix to duplicate headers to make them unique.

    Args:
        headers (list): List of column headers.

    Returns:
        list: List of unique column headers.
    """
    seen = {}
    unique_headers = []
    for header in headers:
        if header in seen:
            seen[header] += 1
            unique_headers.append(f"{header}_{seen[header]}")
        else:
            seen[header] = 1
            unique_headers.append(header)
    return unique_headers


def process_pdfs_in_folder():
    """Processes all PDFs in the specified project range and directory."""
    global core_originals, core_addendums, total_pdfs_accessed, total_pdfs_scraped, total_pdfs_skipped

    # Ensure the log file directory exists
    os.makedirs(os.path.dirname(LOG_FILE_PATH), exist_ok=True)

    with open(LOG_FILE_PATH, 'w') as log_file:
        for project_id in PROJECT_RANGE:
            project_path = os.path.join(BASE_DIRECTORY, str(project_id), "03_phase_2_study")
            if not os.path.exists(project_path):
                missing_projects.add(project_id)
                print(f"Project path does not exist: {project_path}", file=log_file)
                continue

            project_scraped = False  # Flag to track if any PDF in the project was scraped
            base_data_extracted = False
            base_data = pd.DataFrame()

            

            # **START OF CHANGES**
            # Separate PDFs into originals and addendums
            list_pdfs = [pdf for pdf in os.listdir(project_path) if pdf.endswith(".pdf")]
            originals = []
            addendums = []
            for pdf_name in list_pdfs:
                pdf_path = os.path.join(project_path, pdf_name)
                if is_addendum(pdf_path, log_file):
                    addendums.append(pdf_name)
                else:
                    originals.append(pdf_name)
            # **END OF CHANGES**

            # **START OF CHANGES**
            # Process original PDFs first
            for pdf_name in originals:
                
                pdf_path = os.path.join(project_path, pdf_name)
                total_pdfs_accessed += 1

                is_add = is_addendum(pdf_path, log_file)

                # Check if PDF has 'Network Upgrade Type' column
                if has_network_upgrade_type_column(pdf_path, log_file):
                    style_n_pdfs.append(pdf_name)
                    print(f"Skipping PDF: {pdf_name} from Project {project_id} (Style N)", file=log_file)
                    # Still check if original has table7
                    has_table7 = check_has_table7(pdf_path)
                    original_has_table7[project_id] = has_table7
                    continue  # Skip processing this PDF

                print(f"Accessing Original PDF: {pdf_name} from Project {project_id}", file=log_file)
                original_pdfs.append(pdf_name)

                try:
                    has_table7 = check_has_table7(pdf_path)
                    original_has_table7[project_id] = has_table7

                    if not has_table7:
                        skipped_pdfs.append(pdf_name)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 7)", file=log_file)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 7)")
                        total_pdfs_skipped += 1
                        continue

                    if not base_data_extracted:
                        # Extract base data from original PDF
                        base_data = extract_base_data(pdf_path, project_id, log_file)
                        base_data_extracted = True
                        print(f"Extracted base data from original PDF: {pdf_name}", file=log_file)

                    # Extract Table 7 and merge
                    df = extract_table7_and_replace_none(pdf_path, project_id, log_file, is_addendum=False)
                    if not df.empty:
                        core_originals = pd.concat([core_originals, df], ignore_index=True)
                        scraped_pdfs.append(pdf_name)
                        scraped_projects.add(project_id)
                        project_scraped = True
                        total_pdfs_scraped += 1
                        print(f"Scraped PDF: {pdf_name} from Project {project_id}")
                    else:
                        skipped_pdfs.append(pdf_name)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)")
                        total_pdfs_skipped += 1

                except Exception as e:
                    skipped_pdfs.append(pdf_name)
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}", file=log_file)
                    print(traceback.format_exc(), file=log_file)
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}")
                    total_pdfs_skipped += 1
            # **END OF CHANGES**

            # **START OF CHANGES**
            # Then process addendum PDFs
            for pdf_name in addendums:
                pdf_path = os.path.join(project_path, pdf_name)
                total_pdfs_accessed += 1
                is_add = is_addendum(pdf_path, log_file)

                # Check if PDF has 'Network Upgrade Type' column
                if has_network_upgrade_type_column(pdf_path, log_file):
                    style_n_pdfs.append(pdf_name)
                    print(f"Skipping PDF: {pdf_name} from Project {project_id} (Style N)", file=log_file)
                    continue  # Skip processing this PDF

                print(f"Accessing Addendum PDF: {pdf_name} from Project {project_id}", file=log_file)
                addendum_pdfs.append(pdf_name)

                try:
                    has_table7 = check_has_table7(pdf_path)

                    if not has_table7:
                        if original_has_table7.get(project_id, False):
                            # Attempt to scrape alternative tables
                            table7_data = scrape_alternative_tables(pdf_path, log_file, [
                                "PTO",
                                "Reliability Network Upgrade",
                                "Area Delivery Network Upgrade",
                                "Local Delivery Network",
                                "Other Potential Network Upgrade",
                                "Area Delivery Network Upgrades",
                                "Conditionally Assigned Network Upgrades",
                                 
                            ])
                            if not table7_data.empty:
                                # Merge base data with Table 7 data
                                merged_df = pd.concat([base_data] * len(table7_data), ignore_index=True)
                                merged_df = pd.concat([merged_df, table7_data], axis=1, sort=False)
                                core_addendums = pd.concat([core_addendums, merged_df], ignore_index=True)
                                scraped_pdfs.append(pdf_name)
                                scraped_projects.add(project_id)
                                project_scraped = True
                                total_pdfs_scraped += 1
                                print(f"Scraped Addendum PDF: {pdf_name} from Project {project_id}")
                            else:
                                skipped_pdfs.append(pdf_name)
                                print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No relevant tables found)", file=log_file)
                                print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No relevant tables found)")
                                total_pdfs_skipped += 1
                        else:
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No Table 7 and original does not have Table 7)", file=log_file)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (No Table 7 and original does not have Table 7)")
                            total_pdfs_skipped += 1
                        continue

                    if not is_add and not base_data_extracted:
                        # Extract base data from original PDF
                        base_data = extract_base_data(pdf_path, project_id, log_file)
                        base_data_extracted = True
                        print(f"Extracted base data from original PDF: {pdf_name}", file=log_file)

                    if is_add and base_data_extracted:
                        # For addendums, use the extracted base data
                        table7_data = extract_table7(pdf_path, log_file, is_addendum=is_add)
                        if table7_data.empty and original_has_table7.get(project_id, False):
                            # Scrape alternative tables
                            table7_data = scrape_alternative_tables(pdf_path, log_file, [
                                "PTO",
                                "Reliability Network Upgrade",
                                "Area Delivery Network Upgrade",
                                "Local Delivery Network",
                                "Other Potential Network Upgrade",
                                "Area Delivery Network Upgrades",
                                "Conditionally Assigned Network Upgrades",
                                "ADNU",
                                "LDNU",
                                "RNU"
                            ])
                        if not table7_data.empty:
                            


                            
                            # Merge base data with Table 7 data
                            merged_df = pd.concat([base_data] * len(table7_data), ignore_index=True)
                            merged_df = pd.concat([merged_df, table7_data], axis=1, sort=False)
                            core_addendums = pd.concat([core_addendums, merged_df], ignore_index=True)
                            scraped_pdfs.append(pdf_name)
                            scraped_projects.add(project_id)
                            project_scraped = True
                            total_pdfs_scraped += 1
                            print(f"Scraped Addendum PDF: {pdf_name} from Project {project_id}")
                        else:
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                            # Optionally, print to ipynb
                            print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)")
                            total_pdfs_skipped += 1
                except Exception as e:
                    skipped_pdfs.append(pdf_name)
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}", file=log_file)
                    print(traceback.format_exc(), file=log_file)
                    # Optionally, print to ipynb
                    print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}")
                    total_pdfs_skipped += 1
            # **END OF CHANGES**

            # After processing all PDFs for this project, check if any PDF was scraped
            if not project_scraped and os.path.exists(project_path):
                skipped_projects.add(project_id)

    # Rest of the code remains unchanged...


    # After processing all PDFs, save to CSV
    save_to_csv(core_originals, OUTPUT_CSV_PATH_ORIGINAL, "originals")
    save_to_csv(core_addendums, OUTPUT_CSV_PATH_ADDENDUM, "addendums")

    # Calculate total projects processed
    total_projects_processed = len(scraped_projects) + len(skipped_projects)

    # Print summary to ipynb
    print("\n=== Scraping Summary ===")
    print(f"Total Projects Processed: {total_projects_processed}")
    print(f"Total Projects Scraped: {len(scraped_projects)}")
    print(f"Total Projects Skipped: {len(skipped_projects)}")
    print(f"Total Projects Missing: {len(missing_projects)}")
    print(f"Total PDFs Accessed: {total_pdfs_accessed}")
    print(f"Total PDFs Scraped: {total_pdfs_scraped}")
    print(f"Total PDFs Skipped: {total_pdfs_skipped}")

    print("\nList of Scraped Projects:")
    print(sorted(scraped_projects))

    print("\nList of Skipped Projects:")
    print(sorted(skipped_projects))

    print("\nList of Missing Projects:")
    print(sorted(missing_projects))

    print("\nList of Scraped PDFs:")
    print(scraped_pdfs)

    print("\nList of Skipped PDFs:")
    print(skipped_pdfs)

    print("\nList of Addendum PDFs:")
    print(addendum_pdfs)

    print("\nList of Original PDFs:")
    print(original_pdfs)

    print("\nList of Style N PDFs (Skipped due to 'Network Upgrade Type'):")
    print(style_n_pdfs)

    print("\nTotal Number of Style N PDFs:", len(style_n_pdfs))

    print("\nNumber of Original PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in original_pdfs]))
    print("Number of Addendum PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in addendum_pdfs]))

def save_to_csv(df, output_csv_path, data_type):
    """Cleans the DataFrame and saves it to a CSV file."""
    if df.empty:
        print(f"No data to save for {data_type}.")
        return

    # Clean up the entire DataFrame by cleaning string cells
    df = df.applymap(clean_string_cell)

    # Drop rows that contain specific phrases (e.g., "Type of Upgrade")
    df = df[~df.apply(lambda row: contains_phrase(row, "Type of Upgrade"), axis=1)]

    # Reorder columns as specified
    df = reorder_columns(df)
    print(f"\nColumns reordered for {data_type} as per specification.")

    # Ensure q_id is numeric for sorting, replace missing values with None
    if 'q_id' in df.columns:
        df['q_id'] = pd.to_numeric(df['q_id'], errors='coerce')

    # Save the DataFrame to CSV
    try:
        df.to_csv(output_csv_path, index=False)
        print(f"\nData successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving {data_type} data to CSV: {e}")
        print(traceback.format_exc())

def main():
    """Main function to execute the PDF scraping process."""
    process_pdfs_in_folder()

if __name__ == "__main__":
    main()


Scraped PDF: C14_Q1832_GoalLineReliability_PhII_01-30-2024.pdf from Project 1832
Skipped PDF: QC14PII-DCRT-Eastern-Q2022-TOT1101-Socorro Peak-Appendix A-Attachment 1a.pdf from Project 2022 (No Table 7)
Skipped PDF: QC14PII-SCE-Eastern-Q2022 TOT1101-Socorro Peak Solar-Appendix A.pdf from Project 2022 (No Table 7)
Skipped PDF: QC14PII-SCE-Eastern-Q2022 TOT1101-Socorro Peak Solar-Appendix A-Attachment 2b.pdf from Project 2022 (No Table 7)
Skipped PDF: QC14PII-DCRT-Eastern-Q2022-TOT1101-Socorro Peak-Appendix A.pdf from Project 2022 (No Table 7)
Skipped PDF: QC14PII-SCE-Eastern-Q2022 TOT1101-Socorro Peak Solar-Appendix A-Attachment 1b.pdf from Project 2022 (No Table 7)
Skipped PDF: QC14PII-DCRT-Eastern-Q2022-TOT1101-Socorro Peak-Appendix A-Attachment 2a.pdf from Project 2022 (No Table 7)
Skipped Addendum PDF: QC14PII-DCRT-Eastern-Q2022-TOT1101-Socorro Peak-Appendix A-Attachment 1a_SLD (SH 2 OF SH 2).pdf from Project 2022 (No Table 7 and original does not have Table 7)
Skipped Addendum PDF: 

/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_89910/3201901607.py:1159: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clean_string_cell)
/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_89910/3201901607.py:1159: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clean_string_cell)
